# Wonderful Hash

> CRYPTO | 340
> 
> My hash function is composed of AES, DES and RC4. It is Wonderful!
> 
> `nc wonderful-hash.chal.acsc.asia 10217`
> 
> Attachment: wonderful_hash.tar.gz (md5 0ee465ff6b0833676da8c8cc799747c2)

- There is a weird hash function in the python code given that:
  - Pads the input message and splitting into 16 byte blocks
  - Passes each block through `block_hash` to produce 6 bytes of output
  - XOR-s the hashes of all block to produce a final hash

In [1]:
from Crypto.Cipher import AES, ARC4, DES

BLOCK = 16

def bxor(a, b):
  res = [c1 ^ c2 for (c1, c2) in zip(a, b)]
  return bytes(res)

# null blocks pulled out from original code to increase speed
aes_seed = b"\x00" * AES.block_size
arc4_seed = b"\x00" * DES.key_size
des_seed = b"\x00" * DES.block_size

def block_hash(data):
  data = AES.new(data, AES.MODE_ECB).encrypt(aes_seed)
  data = ARC4.new(data).encrypt(arc4_seed)
  data = DES.new(data, DES.MODE_ECB).encrypt(des_seed)
  return data[:-2]

def hash(data):
  length = len(data)
  if length % BLOCK != 0:
    pad_len = BLOCK - length % BLOCK
    data += bytes([pad_len] * pad_len)
    length += pad_len
  block_cnt = length // BLOCK
  blocks = [data[i * BLOCK:(i + 1) * BLOCK] for i in range(block_cnt)]
  res = b"\x00" * BLOCK
  for block in blocks:
    res = bxor(res, block_hash(block))
  return res

- The python service can run an arbitary command in shell
- To change the command string, the new command must have the same length of `417` and hash of `152d18d3ed93` as the original

In [2]:
import string

cmd = (b"echo 'There are a lot of Capture The Flag (CTF) competitions in "
       b"our days, some of them have excelent tasks, but in most cases "
       b"they're forgotten just after the CTF finished. We decided to make"
       b" some kind of CTF archive and of course, it'll be too boring to "
       b"have just an archive, so we made a place, where you can get some "
       b"another CTF-related info - current overall Capture The Flag team "
       b"rating, per-team statistics etc'")

def check(cmd, new_cmd):
  if len(cmd) != len(new_cmd):
    return False
  if hash(cmd) != hash(new_cmd):
    return False
  for c in new_cmd:
    if chr(c) not in string.printable:
      return False
  return True

original_cmd_hash = hash(cmd)

len(cmd), original_cmd_hash.hex()

(417, '152d18d3ed93')

- Our new command can look like
  - Head portion to print flag `cat flag;`
  - Command to "swallow" the rest of the command text `echo  ` (2 spaces, total 1 block so far)
  - Some extra padding to leave an even number of remaining blocks `aaaaaaaaaaaaaaaa` (16 `a`s)
  - 2 blocks to will brute force and align hash
  - Remaining even number of blocks which will we fill with spaces
    - All the block's hashes will be the same
    - Because there is an even number, the hash will cancel out after XOR and we can ignore
  - Tail to pad to ensure correct padding ` ` (1 space)

In [3]:
payload_head = b"cat flag; echo  " + b"a" * BLOCK
payload_brute = b"?" * BLOCK * 2
payload_blank = b" " * BLOCK * 22
payload_tail = b" "

assert(len(payload_head) % BLOCK == 0)
assert(len(payload_tail) == len(cmd) % BLOCK)
assert(len(payload_head + payload_brute + payload_blank + payload_tail) == len(cmd))

- The remainder of the crafted payload has hash `abbcbb066edf`
- To make the final command match original hash, we need to find two blocks which will eventually produce hash `abbcbb066edf`

In [4]:
before_brute_hash = hash(payload_head + payload_blank + payload_tail)
brute_goal_hash = bxor(original_cmd_hash, before_brute_hash)

print(f"""
Hash of original cmd    : {original_cmd_hash.hex()}
Hash of head and tail   : {before_brute_hash.hex()}
Hash required for brute : {brute_goal_hash.hex()}
""".strip())

Hash of original cmd    : 152d18d3ed93
Hash of head and tail   : be91a3d5834c
Hash required for brute : abbcbb066edf


- It is not possible to bryte force 2.8e14 hashes to find one hash that will match the goal
- However, since we have 2 blocks to work with, we can do a "meet in the middle" strategy
  - We precompute 1.7e7 1-block hashes
  - For each hash `h_a`, to produce the goal we will need another block with 1-block hash `h_b = bxor(h_a, goal)` to obtain our goal hash
  - If `h_b` is among the precomputed hashes, we have our desired blocks of text

In [5]:
print(f"""
2 ** (6 * 8)      = {2 ** (6 * 8):e}
2 ** (6 * 8 // 2) = {2 ** (6 * 8 // 2):e}
""".strip())

2 ** (6 * 8)      = 2.814750e+14
2 ** (6 * 8 // 2) = 1.677722e+07


- For redundancy, we precompte 5e7 hashes to increase the chances of finding our desired hash
- Run with `process_map` to calculate hashes with multiprocessing for speedup
  - This took 10 minutes on a 8-core cloud data analysis kernel
- Time to brute force!

In [6]:
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import process_map

sm_max = 10 ** 3
lg_max = 5 * 10 ** 4

def brute_hash(lg):
  res = []
  for x in range(lg * sm_max, (lg + 1) * sm_max):
    h_x = block_hash(("@" + str(x)).zfill(16).encode())
    res.append((h_x, x))
  return res

ACTUAL = False

if ACTUAL:
  # This was run on the cloud kernel
  precomp = process_map(brute_hash, range(lg_max), max_workers=8, chunksize=10)
  cache = {
    a: b
    for i in precomp for a, b, in i
  }
else:
  # Important results exported from cloud kernel
  cache = {
    hash(("@" + str(13698420)).zfill(16).encode()): 13698420,
    hash(("@" + str(46842521)).zfill(16).encode()): 46842521
  }

# Would be around 5e7, but when I ran this it was 5 less due to actual collision
len(cache) 

2

- Now find which strings to use for a collision

In [7]:
# This took around 50s on the cloud kernel
for h, x in tqdm(cache.items()):
    if bxor(brute_goal_hash, h) in cache:
        y = cache[bxor(brute_goal_hash, h)]
        print(x, y)
        break

  0%|          | 0/2 [00:00<?, ?it/s]

13698420 46842521


- Now we sanity check and make sure that the forged command matches

In [8]:
a_soln = 13698420
b_soln = 46842521

soln = (
  payload_head +
  ("@" + str(a_soln)).zfill(16).encode() +
  ("@" + str(b_soln)).zfill(16).encode() + 
  payload_blank +
  payload_tail
)

print(f"""
Original hash : {hash(cmd).hex()}
Original len  : {len(cmd)}
Forged cmd    : {soln.decode()[:70]}...
Forged hash   : {hash(soln).hex()}
Forged len    : {len(soln)}
Check         : {check(soln, cmd)}
""")


Original hash : 152d18d3ed93
Original len  : 417
Forged cmd    : cat flag; echo  aaaaaaaaaaaaaaaa0000000@136984200000000@46842521      ...
Forged hash   : 152d18d3ed93
Forged len    : 417
Check         : True



- Now connect to service and claim flag!

In [9]:
from pwn import *

r = remote("wonderful-hash.chal.acsc.asia", 10217)
r.recvuntil("> ")
r.sendline("S")
r.sendline(soln)
r.recvuntil("> ")
r.sendline("E")
flag = r.recv().decode()

print(flag)

[x] Opening connection to wonderful-hash.chal.acsc.asia on port 10217
[x] Opening connection to wonderful-hash.chal.acsc.asia on port 10217: Trying 35.200.79.53
[+] Opening connection to wonderful-hash.chal.acsc.asia on port 10217: Done
ACSC{M1Tm_i5_FunNY_But_Painfu1}



Flag: `ACSC{M1Tm_i5_FunNY_But_Painfu1}`